In [3]:
# app_id = "686a33c3102ade681afb873f"
# key_id = "app_686a3581cf18a346c80dde1e"


from zenpy import Zenpy
from zenpy.lib.api_objects import Ticket, User, Organization, Group, Comment, Attachment
import os
import json
import requests
from dotenv import load_dotenv, find_dotenv
load_dotenv()


creds = { 
  'email' : os.getenv('ZENDESK_EMAIL'),
  'token' : os.getenv('ZENDESK_TOKEN'),
  'subdomain': os.getenv('ZENDESK_SUBDOMAIN')
}

zenpy_client = Zenpy(**creds)


In [4]:
# Retrieve all users and filter for agents
agents = zenpy_client.users(role='agent') 

for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}")


Agent ID: 36877073033233, Name: tarunpingle@gmail.com
Agent ID: 36884915082129, Name: chinmay thorat
Agent ID: 36885006420881, Name: aditya pandulkar
Agent ID: 36900490423441, Name: prasad kandalkar


In [7]:


# Define the assignee's identifier (email or ID)
assignee_identifier = 'adityapandukar2@gmail.com'  

# Search for users based on the email
users = zenpy_client.users.search(f'email:{assignee_identifier}')

# Check if the search returned any users
if users:
    print(f"Assignee with identifier '{assignee_identifier}' exists in Zendesk.")
else:
    # Create a new user with the "agent" role
    new_agent = zenpy_client.users.create(User(name='aditya pandulkar', email='adityapandukar2@gmail.com', role='agent'))
    print(f"Assignee with email '{assignee_identifier}' not found. Creating a new agent with this email.")
    print(f"Newly created agent ID: {new_agent.id}")




Assignee with identifier 'adityapandukar2@gmail.com' exists in Zendesk.


In [8]:
# If users are found, get the ID of the first user (assuming only one user with this email)
assignee_id = 36885006420881
print(f"Assignee with email '{assignee_identifier}' found. User ID: {assignee_id}")

# Create a new ticket and assign it using the assignee_id
zenpy_client.tickets.create(Ticket(subject="wifi is not working", description="wifi not working at all", assignee_id=assignee_id))
print(f"Ticket created and assigned to user ID {assignee_id}.")

Assignee with email 'adityapandukar2@gmail.com' found. User ID: 36885006420881
Ticket created and assigned to user ID 36885006420881.


In [9]:

# Retrieve all users
all_users = zenpy_client.users()

# Filter for users with the 'end-user' role
customers = [user for user in all_users if user.role == 'end-user']

# Now you have a list of all your customers (end-users)
for customer in customers:
    print(f"Customer Name: {customer.name}, Email: {customer.email}")

Customer Name: lavnya patel, Email: lavnyagpattil@gmail.com
Customer Name: Customer, Email: example@zendesk.com
Customer Name: bob, Email: ckhairnar@gmail.com
Customer Name: anil kumar, Email: anilkumar@gmail.com


In [12]:

# Define the new customer's details
new_customer_name = 'anil kumar'
new_customer_email = 'anilkumar@gmail.com'

# Create a new User object with the 'end-user' role
new_customer = User(name=new_customer_name, email=new_customer_email, role='end-user')

# Create the new user in Zendesk
created_customer = zenpy_client.users.create(new_customer)

# Print details of the newly created customer
print(f"Customer '{created_customer.name}' created with ID: {created_customer.id}")

APIException: {"error": "RecordInvalid", "description": "Record validation errors", "details": {"email": [{"description": "Email: anilkumar@gmail.com is already being used by another user", "error": "DuplicateValue"}]}}

In [13]:

# Search for the user by email
existing_users = zenpy_client.users.search(f'email:{new_customer_email}')

# Check if any users were found with that email
if existing_users:
    print(f"User with email '{new_customer_email}' already exists.")
    # You can access the existing user object here if needed
    existing_user = existing_users.id  # Or users[0].id if you expect multiple matches
    print(f"Existing user ID: {existing_user}")
else:
    # Create a new User object with the 'end-user' role
    new_customer = User(name=new_customer_name, email=new_customer_name, role='end-user')

    # Create the new user in Zendesk
    created_customer = zenpy_client.users.create(new_customer)
    print(f"User '{created_customer.name}' created with ID: {created_customer.id}")

User with email 'anilkumar@gmail.com' already exists.


AttributeError: 'ZendeskResultGenerator' object has no attribute 'id'

In [14]:

# Define the customer (requester) and assignee (agent) emails
customer_email = 'anilkumar@gmail.com'
assignee_email = 'adityapandukar2@gmail.com'  

# --- Get the customer's ID ---
# Search for the customer by email
customer_users = zenpy_client.users.search(f'email:{customer_email}')

# Check if the customer exists and get their ID
if customer_users:
    # Iterate through the generator and get the ID of the first user found
    customer_id = next(customer_users).id  # Use next() to get the first item from the generator
    print(f"Customer with email '{customer_email}' found. User ID: {customer_id}")
else:
    print(f"Customer with email '{customer_email}' not found. Please ensure they exist in Zendesk.")
    # You might want to handle this case, e.g., create the customer or raise an error
    customer_id = None # Set to None if the customer doesn't exist


# --- Get the assignee's ID ---
# Search for the assignee by email
assignee_users = zenpy_client.users.search(f'email:{assignee_email}')

# Check if the assignee exists and get their ID
if assignee_users:
    # Iterate through the generator and get the ID of the first user found
    assignee_id = next(assignee_users).id  # Use next() to get the first item from the generator
    print(f"Assignee with email '{assignee_email}' found. User ID: {assignee_id}")
else:
    print(f"Assignee with email '{assignee_email}' not found. Please ensure they exist in Zendesk and are an agent.")
    # You might want to handle this case, e.g., raise an error or assign to a default agent/group
    assignee_id = None # Set to None if the assignee doesn't exist

# --- Create the ticket if both customer and assignee are found ---
if customer_id and assignee_id:
    # Create the Ticket object, setting requester_id and assignee_id
    new_ticket = Ticket(
        subject="Issue with existing customer and assignee",
        description="Ticket created for an existing customer and assigned to an existing agent.",
        requester_id=customer_id,  # Set the requester using the customer's ID
        assignee_id=assignee_id  # Set the assignee using the agent's ID
    )

    # Create the new ticket in Zendesk
    created_ticket = zenpy_client.tickets.create(new_ticket)

    print(f"Ticket created successfully with ID: {created_ticket.ticket.id}")
else:
    print("Could not create ticket. Either the customer or the assignee was not found.")


Customer with email 'anilkumar@gmail.com' found. User ID: 36885110559889
Assignee with email 'adityapandukar2@gmail.com' found. User ID: 36885006420881
Ticket created successfully with ID: 12


In [15]:

# Retrieve all tickets
tickets = zenpy_client.tickets()

# Iterate through each ticket
for ticket in tickets:
    print(f"Ticket ID: {ticket.id}, Subject: {ticket.subject}, description: {ticket.description}")

    # Get assignee details if assigned
    if ticket.assignee_id:
        try:
            assignee = zenpy_client.users(id=ticket.assignee_id)
            print(f"  Assignee: {assignee.name}, Email: {assignee.email}")
        except Exception as e:
            print(f"  Could not retrieve assignee details for ID {ticket.assignee_id}: {e}")

    # Get requester details
    try:
        requester = zenpy_client.users(id=ticket.requester_id)
        print(f"  Requester: {requester.name}, Email: {requester.email}")
    except Exception as e:
        print(f"  Could not retrieve requester details for ID {ticket.requester_id}: {e}")

    print("-" * 20)  # Separator for readability

Ticket ID: 1, Subject: SAMPLE: How does Zendesk work, description: Hello, let's see how you or your agents can easily respond to and solve tickets.

Feel free to email additional customer test inquiries to **support@creation-58059.zendesk.com**.

But first, let's start by solving one ticket.

Your Zendesk Team
  Assignee: shankar mahadeavan, Email: mar.i.n.esantos.00.5@googlemail.com
  Requester: Customer, Email: example@zendesk.com
--------------------
Ticket ID: 2, Subject: creating my first ticket, description: No description for the first ticket
  Requester: shankar mahadeavan, Email: mar.i.n.esantos.00.5@googlemail.com
--------------------
Ticket ID: 3, Subject: creating my first ticket, description: No description for the first ticket
  Requester: shankar mahadeavan, Email: mar.i.n.esantos.00.5@googlemail.com
--------------------
Ticket ID: 4, Subject: Python API Test Ticket One, description: Demonstrating creating a ticket with the Zendesk API using Python
  Assignee: tarunpingl

In [1]:
from evaluation import ITTicketEvaluator
import os
model_id = os.getenv('HF_MODEL_ID')
api_token = os.getenv('HF_TOKEN')

ticket_evaluator = ITTicketEvaluator(model_id=model_id,api_token=api_token)

ticket_desc = ["The wifi is not working at all. I have tried restarting my computer and router, but it still doesn't connect. Please help me resolve this issue as soon as possible."]
result = ticket_evaluator.classify_with_custom_classifier(ticket_desc)
result


Using device: cuda
Using device: cuda
✅ Custom classifier initialized successfully

=== Custom Classifier Evaluation ===
Classifying 1 queries...
Processing 1/1: The wifi is not working at all. I have tried resta...


[{'query': "The wifi is not working at all. I have tried restarting my computer and router, but it still doesn't connect. Please help me resolve this issue as soon as possible.",
  'department': 'IT Support',
  'priority': 'high',
  'method': 'custom_classifier'}]

In [33]:
for item in result:
    query = item.get('query')
    department = item.get('department')
    priority = item.get('priority')
    method = item.get('method')

In [34]:

# Define the customer (requester) and assignee (agent) emails
customer_email = 'anilkumar@gmail.com'
assignee_email = 'adityapandukar2@gmail.com'  

# --- Get the customer's ID ---
# Search for the customer by email
customer_users = zenpy_client.users.search(f'email:{customer_email}')

# Check if the customer exists and get their ID
if customer_users:
    # Iterate through the generator and get the ID of the first user found
    customer_id = next(customer_users).id  # Use next() to get the first item from the generator
    print(f"Customer with email '{customer_email}' found. User ID: {customer_id}")
else:
    print(f"Customer with email '{customer_email}' not found. Please ensure they exist in Zendesk.")
    # You might want to handle this case, e.g., create the customer or raise an error
    customer_id = None # Set to None if the customer doesn't exist


# --- Get the assignee's ID ---
# Search for the assignee by email
assignee_users = zenpy_client.users.search(f'email:{assignee_email}')

# Check if the assignee exists and get their ID
if assignee_users:
    # Iterate through the generator and get the ID of the first user found
    assignee_id = next(assignee_users).id  # Use next() to get the first item from the generator
    print(f"Assignee with email '{assignee_email}' found. User ID: {assignee_id}")
else:
    print(f"Assignee with email '{assignee_email}' not found. Please ensure they exist in Zendesk and are an agent.")
    # You might want to handle this case, e.g., raise an error or assign to a default agent/group
    assignee_id = None # Set to None if the assignee doesn't exist

# --- Create the ticket if both customer and assignee are found ---
if customer_id and assignee_id:
    # Create the Ticket object, setting requester_id and assignee_id
    new_ticket = Ticket(
        subject="wifi is not working",
        description=query,
        requester_id=customer_id,  # Set the requester using the customer's ID
        assignee_id=assignee_id,  # Set the assignee using the agent's ID
        priority=priority,  # Set the priority based on the classification
        department=department,  # Set the department based on the classification
    )

    
    # Create the new ticket in Zendesk
    created_ticket = zenpy_client.tickets.create(new_ticket)

    print(f"Ticket created successfully with ID: {created_ticket.ticket.id}")
else:
    print("Could not create ticket. Either the customer or the assignee was not found.")


Customer with email 'anilkumar@gmail.com' found. User ID: 36885110559889
Assignee with email 'adityapandukar2@gmail.com' found. User ID: 36885006420881
Ticket created successfully with ID: 13
